# Determinants of Household Food Insecurity Amongst Urban Household in Nigeria Using MICS6 Dataset:
### A multilevel multinorminal logistic regression  and mixed effect analysis.


**Objectives**
1. To assess factors associated with household food insecurity in Nigeria



### Data source and study population
This study is based on quantitative cross-sectional data derived from the Nigeria 2021 Multiple Indicator Cluster Survey (MICS6), which is a nationally representative survey that collects sociodemographic and health indicators from both household, males and females aged 15–49 years. 

The survey utilized a multistage stratified cluster sampling approach that employed a probability proportional to size to select enumeration areas in the first stage based on the 2006 Population and Housing Census of the Federal Republic of Nigeria (NPHC). In the second stage, 20 households were randomly selected within each enumeration area.

### Load the required libraries

In [1]:
# Load the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import functools
import operator
pd.set_option("display.max_columns", 400)


#### Read the data into a dataframe


In [2]:
# Household information panel
file_path = "../../../data/mics6ng/raw"

hh = pd.read_spss(f"{file_path}/hh.sav")

In [3]:
# Women(15-49 years)
wm = pd.read_spss(f"{file_path}/wm.sav")

In [4]:
# Men(15-49 years)
mn = pd.read_spss(f"{file_path}/mn.sav")

In [5]:
# Data collected at the children's level 
ch = pd.read_spss(f"{file_path}/ch.sav")

## **Data Pre-Processing Pipeline.**

### MISC data files
Each of the data files contains data collected in the relevant questionnaires. Some variables considered to be of critical importance for the analysis are included in all data files, regardless of the questionnaire used to collect the information.

All information required for this analysis is not present in one single data file. There is need for `MICS data` files to be combined or merged. 

In [6]:
# Turn the column names to lowercase
def rename_colnames_tolower(df):
    """Rename the column names to lower case"""
    return df.rename(str.lower, axis="columns")

### **Select Relevant Variables from `hh.sav`**
* Format the variables names to lowercase.
* Identify the relevant columns.
* Create a dictionary of column names.
* Subset with the selected variables
* Filter the household data by consent `hh12`, **MICS** or NICS `survey`

In [8]:
# Turn to lower the column names
hh = rename_colnames_tolower(hh)

# household columns
hh_cols = ["hh1", "hh2", "hh6","hh7", "zone", 
           # consent
           "hh12",
           # "hh8", "
           "hh48", "hh49", "hh50", "hh51","hh52",
           "hhsex", "hhage", "helevel", "hc14"]


# Household head have and have not
hh_livestock_cols = ["hc15", "hc16", "hc17"] + [col for 
                                        col in hh.columns if col.startswith("hc18")]

# Household worried about not having enough food (FIES)
hh_fies_cols = [col for col in hh.columns if col.startswith("fe")]


# Source of water 
hh_water_cols = ["ws1", "ws2", "ws3"]

#Sanitation
hh_toilet_cols = ["ws11", "ws14", "ws15", "ws16"]


# hh wealth index columns
hh_windex_cols = ["windex5mics","windex5umics"]


# hh survey design columns
hh_meta_data_cols = ["hhweightmics", "psu", "stratum", "survey"]


hh_final_cols = functools.reduce(operator.add, [hh_cols, hh_windex_cols, 
                                                hh_livestock_cols, hh_water_cols, 
                                                hh_fies_cols, hh_meta_data_cols])

print(f"Total number of variables selected: {len(hh_final_cols)}")

Total number of variables selected: 67


In [9]:
# hh columns to rename
hh_dict = {
    "ws11": "toilet_type",
    "ws14": "toilet_location",
    "ws15": "toilet_num_hh_using",
    "ws16": "toilet_shared",
    "ws1": "water_source",
    "ws2": "other_water_source",
    "ws3": "water_source_location",
    "hh6": "area", 
    "hh7": "state",
    "hh12": "consent",
    "hc14": "hh_own_dwelling",
    "hc15": "hh_agricultural_land",
    "hc16": "hh_ag_land_size",
    "hc17": "hh_own_animal",
    "hh48": "hh_members_num",
    "hh49" : "num_of_women_15_49",
    "hh50" : "num_of_men_15_49",
    "hh51" : "num_under_5_child",
    "hh52" : "num_5_17_child", 
    # Wealth index
    "windex5umics": "urban_wi_quintile_mics",
    "windex5mics": "wi_quintile_mics",

    
}

In [10]:
# Subset the dataframe
hh_subset = hh[hh_final_cols]

In [11]:
hh_subset = hh_subset.rename(columns=hh_dict)

In [12]:
# Filter the household data by consent consent
print("Dataframe shape filter by consent", hh_subset.shape)
hh_subset = hh_subset.loc[hh_subset["consent"] == "YES"]
print("Dataframe shape after filter by consent", hh_subset.shape)


Dataframe shape filter by consent (41532, 67)
Dataframe shape after filter by consent (39632, 67)


**Observation**
* Of the Data collected at the household level, only 

In [13]:
hh_subset["survey"].value_counts()

survey
MICS    33631
NICS     6001
Name: count, dtype: int64

In [14]:
# Filter by the case source (survey): MICS or NICS survey
print("Dataframe shape filter by MICS", hh_subset.shape)
hh_subset = hh_subset.loc[hh_subset["survey"] == "MICS"].reset_index(drop=True)
print("Dataframe shape after filter by MICS", hh_subset.shape)

Dataframe shape filter by MICS (39632, 67)
Dataframe shape after filter by MICS (33631, 67)


**FIES `(fe1 - fe8c)`**

* Select the variables that contains the prevalence of food insecurity experience scale.
* Transform the values to numerical. That is, map {Yes:1, No:0}.
* Save the resulting variables to dataframe to perform [`Ransch analysis`](https://elearning.fao.org/pluginfile.php/491591/mod_scorm/content/5/story_content/external_files/SDG2.1.2_lesson3.pdf)

In [15]:
# Household worried about not having enough food (FIES)
import re

# Select primary fies question only
fies_cols = [col for col in hh_subset.columns if re.search(r'^(fe)\d+$', col)]

hh_subset[fies_cols] = hh_subset[fies_cols].apply(lambda x:x.str.upper() == "YES").astype("int8")

hh_subset = hh_subset.assign(fies_score = hh_subset[fies_cols].sum(axis=1))

# Select FIES 
# Create a list of columns including specific columns ("hh1", "hh2", "area", "zone", "hhweightmics") 
# and additional columns from hh_subset that start with "fe"
fies_cols = ["hh1", "hh2", "area", "zone", "hhweightmics"] + [col for col in hh_subset.columns if col.startswith("fe")]

# Select the specified columns from the hh_subset dataframe and save it to a new dataframe fies_df
fies_df = hh_subset[fies_cols]

# Save the fies_df dataframe to a CSV file named "fies_df.csv" without including row indices
fies_df.to_csv("../processed_data/hh_fies_df.csv", index=False)

# Drop the FIES columns from the hh_subset dataframe
hh_subset = hh_subset.drop(columns=hh_fies_cols, axis=1)


**Agricultural Land `hh_ag_land_size` and `hh_agricultural_land`**

* Replace NaN values with 0.0 in `hh_ag_land_size` if response to `hh_agricultural_land` is 'NO'

In [16]:
# Create a boolean mask for rows where 'hh_agricultural_land' is 'NO'
mask = hh_subset['hh_agricultural_land'] == "NO"

# Replace NaN with 0 in 'hh_ag_land_size' where the mask is True
hh_subset.loc[mask, 'hh_ag_land_size'] = hh_subset.loc[mask, 'hh_ag_land_size'].fillna(0)

hh_subset.head(2)

,hh1,hh2,area,state,zone,consent,hh_members_num,num_of_women_15_49,num_of_men_15_49,num_under_5_child,num_5_17_child,hhsex,hhage,helevel,hh_own_dwelling,wi_quintile_mics,urban_wi_quintile_mics,hh_agricultural_land,hh_ag_land_size,hh_own_animal,hc18a,hc18b,hc18c,hc18d,hc18e,hc18f,hc18g,hc18h,hc18i,hc18k,hc18l,hc18m,hc18n,water_source,other_water_source,water_source_location,hhweightmics,psu,stratum,survey,fies_score
0,1.0,1.0,URBAN,ABIA,South East,YES,4.0,3.0,0.0,0.0,1.0,Female,46.0,Primary,RENT,Fourth,Middle,NO,0.0,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,7
1,1.0,2.0,URBAN,ABIA,South East,YES,2.0,2.0,0.0,0.0,1.0,Female,45.0,Senior secondary,RENT,Fourth,Second,NO,0.0,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,6


**Household Livestock**

* For household who responded to 'NO' to `hc17` replace hh_ls_cols with 0
* Select the household livestock variables.
* Create a new variable `hh_ls_num` (The sum of livestock per household)
* 

In [17]:


cols_to_modify = [col for col in hh_subset.columns if col.startswith("hc18")]

mask = hh_subset["hh_own_animal"].isin(["NO", "NO RESPONSE"])
hh_subset.loc[mask, cols_to_modify] = hh_subset.loc[mask, cols_to_modify].fillna(0.0)

# Get the livestock columns and change as types from category to int. 
# Run this code onces.
hh_ls = hh_subset[cols_to_modify].apply(lambda x:x.cat.codes)

# Add a new column to indication the total livestock per hh
hh_subset["hh_ls_num"] = hh_ls.sum(axis=1)

# 
ls_cols = ["hh_own_animal", "hh1", "hh2", "hhsex", 
           "hh_own_dwelling", "hh_agricultural_land", 
           "hh_ag_land_size"] + \
        [col for col in hh_subset.columns if col.startswith("hc18")]

ls_df = hh_subset[ls_cols]

# Save data to csv
ls_df.to_csv("../processed_data/hh_livestock_dataset.csv", index=False)


# # Drop the other livestock columnes
hh_subset = hh_subset.drop(columns=cols_to_modify)

hh_subset.head()



,hh1,hh2,area,state,zone,consent,hh_members_num,num_of_women_15_49,num_of_men_15_49,num_under_5_child,num_5_17_child,hhsex,hhage,helevel,hh_own_dwelling,wi_quintile_mics,urban_wi_quintile_mics,hh_agricultural_land,hh_ag_land_size,hh_own_animal,water_source,other_water_source,water_source_location,hhweightmics,psu,stratum,survey,fies_score,hh_ls_num
0,1.0,1.0,URBAN,ABIA,South East,YES,4.0,3.0,0.0,0.0,1.0,Female,46.0,Primary,RENT,Fourth,Middle,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,7,0
1,1.0,2.0,URBAN,ABIA,South East,YES,2.0,2.0,0.0,0.0,1.0,Female,45.0,Senior secondary,RENT,Fourth,Second,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,6,0
2,1.0,3.0,URBAN,ABIA,South East,YES,5.0,1.0,0.0,2.0,1.0,Male,41.0,Senior secondary,OWN,Fourth,Middle,NO,0.0,YES,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,5,30
3,1.0,4.0,URBAN,ABIA,South East,YES,4.0,1.0,1.0,0.0,0.0,Male,81.0,None,OWN,Middle,Poorest,YES,2.0,YES,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,7,144
4,1.0,5.0,URBAN,ABIA,South East,YES,5.0,1.0,0.0,0.0,1.0,Male,63.0,Primary,RENT,Fourth,Middle,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,7,0


### **Select Relevant Columns from `mn.sav` Men (15-49 years of age)**

* Format column name to lowercase
* Select variable of interest
* Rename variables of interest
* Filter by consent

In [18]:
# Turn to lower the column names
mn = rename_colnames_tolower(mn)

mn_cols = ["hh1", "hh2", "ln","hh6",
           "mwb4", "mwb5",
           "mmstatus","mma1",
           "mwage",
           "mls1", "mls2", "mls3", "mls4",
           "mwelevel",
           "survey", "mwm9"]

mn_dict = {
    "mwm9": "consent",
    "mls1": "mn_overall_happiness",
    "mls2": "mn_life_satisfaction",
    "mls3": "mn_life_satisfaction_last_year",
    "mls4": "mn_life_satisfaction_expectation",
    "mwb5": "mn_attended_sch",
    "mwelevel": "mn_education",
    "mmstatus": "mn_mstatus",
    "mma1": "mn_currently_married",
    "mwage": "mn_age_cat",
    "mwb4": "mn_age_yrs",
    
}

# Create a subset of the 'mn' DataFrame, 
# including only the columns specified in 'mn_cols'
mn_subset = mn[mn_cols]

# Rename the columns of mn_subset using the key-value pairs in mn_dict
mn_subset = mn_subset.rename(columns=mn_dict)


# Filter out records where consent is not "YES"
mn_subset = mn_subset[mn_subset["consent"] == "YES"].reset_index(drop=True)



In [19]:
mn_subset.shape

(17347, 16)

**Men Life Satisfaction `mn_life_satisfaction`**

In [20]:
mn_subset[["mn_life_satisfaction"]].value_counts(dropna=False)

mn_life_satisfaction
5                       3081
6                       2327
4                       2180
7                       2153
3                       1968
8                       1842
2                       1421
9                        860
1                        746
10                       631
0                        132
NO RESPONSE                6
Name: count, dtype: int64

* The variable doesn't include NaN values
* Replace the value "NO RESPONSE" with NaN


In [21]:
# Replace "NO response" with NaN
mn_subset['mn_life_satisfaction'] = mn_subset['mn_life_satisfaction'].replace("NO RESPONSE", np.nan)

mn_subset['mn_life_satisfaction_num'] = mn_subset['mn_life_satisfaction'].astype(float)

/var/folders/v2/zgr5gbws4qd_qpb553947v7h0000gn/T/ipykernel_4350/6390347.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  mn_subset['mn_life_satisfaction'] = mn_subset['mn_life_satisfaction'].replace("NO RESPONSE", np.nan)


### **Select Relevant Columns from `wm.sav`:  Women (15-49 years of age)**

* Select variable of interest
* Rename variables of interest
* Filter by consent

In [22]:
# Select column of interest from wm.sav

wm = rename_colnames_tolower(wm)

wm_cols = ["hh1", "hh2", "ln","hh6",
           # "wm1", "wm2", "wm3",
           "wb4", "wm9",
           "wage", "wb5", "wb6a","welevel", 
           "ls1","ls2","ls3","ls4",
           "survey", "mstatus", 
           "ma1", 
           "windex5u",
           "wmweight", "stratum"]

wm_dicts  = {
    # "wm1": "w_cluster_num",
    # "wm2": "w_hh_num",
    # "wm3": "w_line_num",
    "wb4": "wm_age_yrs",
    "wm9": "consent",
    "ls1": "wm_overall_happiness",
    "ls2": "wm_life_satisfaction",
    "ls3": "wm_life_satisfaction_last_year",
    "ls4": "wm_life_satisfaction_expectation",
    "wage": "wm_age_cat",
    "wb5": "wm_attended_sch",
    "wb6a": "wm_level_education",
    "mstatus": "wm_mstatus",
    "ma1": "wm_currently_married",
     # Wealth index
    "windex5u": "urban_wi_quintile_mics",

}

# Create a subset of the 'wm' DataFrame, including only the columns specified in 'wm_cols'
wm_subset = wm.loc[:, wm_cols]

# Rename the columns of wm_subset using the key-value pairs in wm_dicts
wm_subset = wm_subset.rename(columns=wm_dicts)

# Filter out records where consent is not "YES"
wm_subset = wm_subset[wm_subset["consent"] == "YES"].reset_index(drop=True)

In [23]:
wm_subset.shape

(38813, 20)

In [24]:
wm_subset.head(2)

,hh1,hh2,ln,hh6,wm_age_yrs,consent,wm_age_cat,wm_attended_sch,wm_level_education,welevel,wm_overall_happiness,wm_life_satisfaction,wm_life_satisfaction_last_year,wm_life_satisfaction_expectation,survey,wm_mstatus,wm_currently_married,urban_wi_quintile_mics,wmweight,stratum
0,1.0,1.0,1.0,URBAN,46.0,YES,45-49,YES,PRIMARY,Primary,SOMEWHAT UNHAPPY,3,WORSENED,BETTER,MICS,Currently married/in union,"YES, CURRENTLY MARRIED",Middle,0.631645,1.0
1,1.0,1.0,2.0,URBAN,22.0,YES,20-24,YES,SENIOR SECONDARY,Senior secondary,NEITHER HAPPY NOR UNHAPPY,4,WORSENED,BETTER,MICS,Never married/in union,"NO, NOT IN UNION",Middle,0.631645,1.0


**Live Satisfaction `wm_life_satisfaction`**

In [25]:
wm_subset['wm_life_satisfaction'].isna().sum()

7

In [26]:
wm_subset['wm_life_satisfaction'].value_counts(dropna=False)

wm_life_satisfaction
5              7010
6              5517
7              5285
8              4959
10             4052
4              3831
9              3475
3              2364
2              1173
1               823
0               249
NO RESPONSE      68
NaN               7
Name: count, dtype: int64

In [27]:
# Replace "NO RESPONSE" with NaN
wm_subset['wm_life_satisfaction'] = wm_subset['wm_life_satisfaction'].replace("NO RESPONSE", np.nan)
wm_subset['wm_life_satisfaction_num'] = wm_subset['wm_life_satisfaction'].astype(float)

wm_subset['wm_life_satisfaction_num'].isna().sum()

/var/folders/v2/zgr5gbws4qd_qpb553947v7h0000gn/T/ipykernel_4350/2384192019.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  wm_subset['wm_life_satisfaction'] = wm_subset['wm_life_satisfaction'].replace("NO RESPONSE", np.nan)


75

**Inferenece**

* `hh1` matches `w_cluster_num`
* `hh2` matches `w_hh_num`
* `hh3` matches `w_line_num`

We can join the `wm_subset` with `hh_subset` with variables a

### **Select Relevant Columns from `ch.sav` : (Under five)**

* Join the children under five years day with the houshold data

In [28]:
# 
ch = rename_colnames_tolower(ch)

ch_cols = ["hh1", "hh2", "ln",
           # "uf1", "uf2", "uf3",
           # mother
            "uf4",
           "melevel",
           #
           "uf10",
           "survey",
           # sex
           "hl4",
           "ub2","cage","cage_6","cage_11", 
           # food groups
           "bd2", "bd3", "bd7d", "bd7e", "bd8a", "bd8b",
           "bd8c", "bd8d", "bd8e", "bd8f", "bd8g", "bd8h", 
           "bd8i", "bd8j", "bd8k", "bd8l", "bd8m", "bd8n",
           # --
           "zone", "hh6",
           "melevel", "stratum", "chweightmics" ]


ch_dicts = {
    "uf4" : "c_mother_ln",
    "melevel": "c_mother_edu_levl",
    "ub2" : "c_age_yr",
    "uf10": "consent",
    "cage": "c_age_month",
    "cage_6" : "c_age_month_cat6",
    "cage_11" : "c_age_month_cat11",
    # -- 
    "hl4" : "c_sex",
    "melevel" : "c_mother_education",
    # Food groups
    "bd2" : "c_breastfed",
    "bd3" : "c_breastfeeding",
    "bd7d" : "c_infant_formula",
    "bd7e" : "c_animal_milk",
    "bd8a" : "c_yogurt",
    "bd8b" : "c_fortified_food",
    "bd8c" : "c_grain",
    "bd8d" : "c_vitA_vegs",
    "bd8e" : "c_roots",
    "bd8f" : "c_green_leafy_vegs",
    "bd8g" : "c_vitA_fruits",
    "bd8h" : "c_other_fruits_veg",
    "bd8i" : "c_organ_meat",
    "bd8j" : "c_meat",
    "bd8k" : "c_eggs",
    "bd8l" : "c_fish",
    "bd8m" : "c_beans_pulse",
    "bd8n" : "c_cheese_f_milk",
    # location
    "hh6": "area",
}


# Create a subset of the 'ch' DataFrame, including only the columns specified in 'wm_cols'
ch_subset = ch.loc[:, ch_cols]

# Rename the columns of ch_subset using the key-value pairs in ch_dicts
ch_subset = ch_subset.rename(columns=ch_dicts)

# Filter out records where consent is not "YES"
ch_subset = ch_subset[ch_subset["consent"] == "YES"]

# Filter out records where survey is not "MICS"
ch_subset = ch_subset[ch_subset["survey"] == "MICS"].reset_index(drop=True)

In [29]:
ch_subset["survey"].value_counts()

survey
MICS    26640
NICS        0
Name: count, dtype: int64

In [30]:
ch_subset.shape

(26640, 35)

**Inference**
After initial preprocessing that includes:
* Filter by survey
* Selecting relevant variables from the man, household and women data
* Filter by consent

We shall proceed with the following dataset

In [31]:
# Check the current status of our dataframes
print("shape of Household:", hh_subset.shape)
print("shape of Women(15-49 years of age):", wm_subset.shape)
print("shape of Men(15-49 years of age):", mn_subset.shape)

print("shape of Children under five:", ch_subset.shape)

shape of Household: (33631, 29)
shape of Women(15-49 years of age): (38813, 21)
shape of Men(15-49 years of age): (17347, 17)
shape of Children under five: (26640, 35)


In [32]:
# Save the children under 5 years old to MDDC folder
ch_subset.to_csv("../../../MDDS/data/children_under_five.csv", index=False)

# Save household data to MISC folder
hh_subset.to_csv("../processed_data/household.csv", index=False)

# Save women data to MISC folder
wm_subset.to_csv("../processed_data/wm_subset.csv", index=False)

# Save men data to MISC folder
wm_subset.to_csv("../processed_data/mn_subset.csv", index=False)

### **Merging instructions**

```
key_variables_dict = {
    "hh1": "Cluster number",
    "hh2": "household number",
    "ln": "Members line number"
}
```


* We will use `HH1` and `HH2` as key variables to merge `ch_subset`, `mn_subset`, `wm_subset` with `hh_subset` dataframe

In [33]:
# Load in the preprocessed dataset
hh_df = pd.read_csv("../processed_data/household.csv")
wm_df = pd.read_csv("../processed_data/wm_subset.csv")
mn_df = pd.read_csv("../processed_data/mn_subset.csv")
ch_df = pd.read_csv("../../../MDDS/data/children_under_five.csv")



#### Merge the household dataframe with women dataframe

In [34]:
# Merge the hh_subset and wn_subset DataFrames on the 
# "hh1 and hh2" key.

# first sort both dataframe nu the key variables

merged_hh_wm = pd.merge(hh_df, 
                        wm_df, 
                        on=["hh1", "hh2"], 
                        how="left",
                       suffixes=("_hh", "_wm"))
merged_hh_wm.shape


(46716, 48)

#### Merge the household dataframe with men dataframe

In [35]:
merged_hh_mn = pd.merge(hh_df, 
                        mn_df, 
                        on=["hh1", "hh2"], 
                        how="left")


merged_hh_mn.shape

(46716, 48)

#### Merge the household dataframe with the men and women dataframe

In [36]:
merged_hh_wm_mn = pd.merge(merged_hh_wm, 
                           mn_subset, 
                           on=["hh1", "hh2"], 
                           how="left",
                          suffixes=("_wm", "_mn")
                          )


In [37]:
merged_hh_wm_mn.head(2)

,hh1,hh2,area,state,zone,consent_hh,hh_members_num,num_of_women_15_49,num_of_men_15_49,num_under_5_child,num_5_17_child,hhsex,hhage,helevel,hh_own_dwelling,wi_quintile_mics,urban_wi_quintile_mics_hh,hh_agricultural_land,hh_ag_land_size,hh_own_animal,water_source,other_water_source,water_source_location,hhweightmics,psu,stratum_hh,survey_hh,fies_score,hh_ls_num,ln_wm,hh6_wm,wm_age_yrs,consent_wm,wm_age_cat,wm_attended_sch,wm_level_education,welevel,wm_overall_happiness,wm_life_satisfaction,wm_life_satisfaction_last_year,wm_life_satisfaction_expectation,survey_wm,wm_mstatus,wm_currently_married,urban_wi_quintile_mics_wm,wmweight,stratum_wm,wm_life_satisfaction_num,ln_mn,hh6_mn,mn_age_yrs,mn_attended_sch,mn_mstatus,mn_currently_married,mn_age_cat,mn_overall_happiness,mn_life_satisfaction,mn_life_satisfaction_last_year,mn_life_satisfaction_expectation,mn_education,survey,consent,mn_life_satisfaction_num
0,1.0,1.0,URBAN,ABIA,South East,YES,4.0,3.0,0.0,0.0,1.0,Female,46.0,Primary,RENT,Fourth,Middle,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,7,0,1.0,URBAN,46.0,YES,45-49,YES,PRIMARY,Primary,SOMEWHAT UNHAPPY,3.0,WORSENED,BETTER,MICS,Currently married/in union,"YES, CURRENTLY MARRIED",Middle,0.631645,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,URBAN,ABIA,South East,YES,4.0,3.0,0.0,0.0,1.0,Female,46.0,Primary,RENT,Fourth,Middle,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,7,0,2.0,URBAN,22.0,YES,20-24,YES,SENIOR SECONDARY,Senior secondary,NEITHER HAPPY NOR UNHAPPY,4.0,WORSENED,BETTER,MICS,Never married/in union,"NO, NOT IN UNION",Middle,0.631645,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### **Long Data to Wide Data**: Create a final DataFrame from `hh, women, men` DataFrames.

By the survey design, the `wm` and `mn` datasets are long format. There are multiple rows for each entity (e.g., cluster, household, household member) across different data point.

Therefore the need to make each row represents a single entity, and different data point are represented by separate columns.

Create a DataFrame where each record represent a single data point.

#### **Pivot the women data `wm_subset`**

We want to connect the record of everyone woman in the `wm` dataset with each record in the `hh`. 

* `hh1` and `hh2` uniquely identifies a household. To identify a women, we index by `hh1`, `hh2` and `ln`.
* Select selected variables of interest

In [85]:
wm_subset.head(2)

,hh1,hh2,ln,hh6,wm_age_yrs,consent,wm_age_cat,wm_attended_sch,wm_level_education,welevel,wm_overall_happiness,wm_life_satisfaction,wm_life_satisfaction_last_year,wm_life_satisfaction_expectation,survey,wm_mstatus,wm_currently_married,urban_wi_quintile_mics,wmweight,stratum,wm_life_satisfaction_num
0,1.0,1.0,1.0,URBAN,46.0,YES,45-49,YES,PRIMARY,Primary,SOMEWHAT UNHAPPY,3,WORSENED,BETTER,MICS,Currently married/in union,"YES, CURRENTLY MARRIED",Middle,0.631645,1.0,3.0
1,1.0,1.0,2.0,URBAN,22.0,YES,20-24,YES,SENIOR SECONDARY,Senior secondary,NEITHER HAPPY NOR UNHAPPY,4,WORSENED,BETTER,MICS,Never married/in union,"NO, NOT IN UNION",Middle,0.631645,1.0,4.0


In [86]:
# Pivot with multiple value columns
df_wm = wm_subset.pivot_table(index=['hh1', 'hh2'], columns='ln', 
                                     values=['wm_attended_sch', 'wm_age_yrs', 'welevel', 
                                             'wm_life_satisfaction_num',
                                             'wm_overall_happiness'], 
                                     aggfunc='last')

df_wm = df_wm.sort_index(axis=1, level=1)
df_wm.columns = [f'{x}_{y:,.0f}' for x,y in df_wm.columns]
df_wm = df_wm.reset_index()

In [87]:
print(f"The number of women surveyed(MICS) in urban area are from {df_wm.shape[0]} unique households")

The number of women surveyed(MICS) in urban area are from 25727 unique households


In [88]:
# Sanity check
# wm_subset[(wm_subset["hh1"]==1.0)&(wm_subset["hh2"]==2.0) & (wm_subset["welevel"].isin(["Senior secondary", "Higher/tertiary"]))]

From the `wm` dataset, we create 5 new variables that could be added to the household dateset as single record

In [89]:
# Number of women that attended any school school
# Count the number of "Yes" by row
df_wm = df_wm.assign(
    hh_wm_attended_sch_num=lambda x:(x.filter(like="wm_attended_sch")=="YES").sum(axis=1),
    hh_wm_mean_age_yrs=lambda x:(x.filter(like="wm_age_yrs")).mean(axis=1),
    hh_wm_attended_atleast_sec_sch =lambda x:(x.filter(like="welevel").isin(["Senior secondary", "Higher/tertiary"])).sum(axis=1),
    hh_wm_mean_life_satisfaction =lambda x:(x.filter(like="wm_life_satisfaction_num")).mean(axis=1),
    hh_wm_overall_happiness=lambda x:(x.filter(like="wm_overall_happiness").isin(["SOMEWHAT HAPPY","VERY HAPPY"])).sum(axis=1),
    
    
)
df_wm.shape

(25727, 152)

In [90]:
# Select specific columns related to women's household data from the dataframe df_wm
df_women = df_wm[["hh1", "hh2", "hh_wm_attended_sch_num", 
                  "hh_wm_mean_age_yrs", "hh_wm_attended_atleast_sec_sch", 
                  "hh_wm_overall_happiness","hh_wm_mean_life_satisfaction"]]


df_women.shape

(25727, 7)

**Inference**

* Created 5 new variables related to women in the household.
* Select the 5 new created variables is to be added to the household dataframe. Each record will be unique for a household

#### **Pivot the Men data `mn_subset`**

We want to connect the record of everyone man in the `mn` dataset with each record in the `hh`. 

* `hh1` and `hh2` uniquely identifies a household. To identify a women, we index by `hh1`, `hh2` and `ln`.
* Select selected variables of interest
* The output dataset would be record of a man in his household

In [91]:
# Pivot with multiple value columns
df_mn = mn_subset.pivot_table(index=['hh1', 'hh2'], columns='ln', 
                                     values=['mn_attended_sch', 'mn_age_yrs', 'mn_education', 
                                             'mn_life_satisfaction_num',
                                             'mn_overall_happiness'], 
                                     aggfunc='last')

df_mn = df_mn.sort_index(axis=1, level=1)
df_mn.columns = [f'{x}_{y:,.0f}' for x,y in df_mn.columns]
df_mn = df_mn.reset_index()

In [92]:
print(f"The number of men surveyed(MICS) are from {df_mn.shape[0]} unique households")


The number of men surveyed(MICS) are from 12036 unique households


In [93]:
# Number of women that attended any school school
# Count the number of "Yes" by row
df_mn = df_mn.assign(
    hh_mn_attended_sch_num=lambda x:(x.filter(like="mn_attended_sch")=="YES").sum(axis=1),
    hh_mn_mean_age_yrs=lambda x:(x.filter(like="mn_age_yrs")).mean(axis=1),
    hh_mn_attended_atleast_sec_sch =lambda x:(x.filter(like="mn_education").isin(["Senior secondary", "Higher/tertiary"])).sum(axis=1),
    hh_mn_mean_life_satisfaction =lambda x:(x.filter(like="mn_life_satisfaction_num")).mean(axis=1),
    hh_mn_overall_happiness=lambda x:(x.filter(like="mn_overall_happiness").isin(["SOMEWHAT HAPPY","VERY HAPPY"])).sum(axis=1),
    
)


In [94]:
# Select specific columns related to women's household data from the dataframe df_wm
df_men = df_mn[["hh1", "hh2", "hh_mn_attended_sch_num", "hh_mn_mean_age_yrs", 
                  "hh_mn_attended_atleast_sec_sch", "hh_mn_mean_life_satisfaction", "hh_mn_overall_happiness"]]


df_men.shape

(12036, 7)

**Inference**

* Created 5 new variables related to adult men in the household.
* Select the 5 new created variables to added to the household data

#### Merge the new subset dataframes to create a final dataset

In [95]:
merged_hh_women = pd.merge(hh_subset, df_women, 
                            on=["hh1", "hh2"], 
                            how="left",
                            suffixes=("_hh", "_wm")
                          )
merged_hh_women.shape

(33631, 34)

### Final Merge for Household Analysis

Having created created some variables of interest from the `mn` and `wm` dataset that is unique for a record in the `hh` dataframe, let merge them together.

In [96]:
final_merged_df = pd.merge(merged_hh_women, df_men, 
         on=["hh1", "hh2"], 
         how="left",
         suffixes=("_hh", "_wm"))

final_merged_df.shape

(33631, 39)

In [99]:
print(final_merged_df.shape)
final_merged_df.head()

(33631, 39)


,hh1,hh2,area,state,zone,consent,hh_members_num,num_of_women_15_49,num_of_men_15_49,num_under_5_child,num_5_17_child,hhsex,hhage,helevel,hh_own_dwelling,wi_quintile_mics,urban_wi_quintile_mics,hh_agricultural_land,hh_ag_land_size,hh_own_animal,water_source,other_water_source,water_source_location,hhweightmics,psu,stratum,survey,fies_score,hh_ls_num,hh_wm_attended_sch_num,hh_wm_mean_age_yrs,hh_wm_attended_atleast_sec_sch,hh_wm_overall_happiness,hh_wm_mean_life_satisfaction,hh_mn_attended_sch_num,hh_mn_mean_age_yrs,hh_mn_attended_atleast_sec_sch,hh_mn_mean_life_satisfaction,hh_mn_overall_happiness
0,1.0,1.0,URBAN,ABIA,South East,YES,4.0,3.0,0.0,0.0,1.0,Female,46.0,Primary,RENT,Fourth,Middle,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,7,0,3.0,28.666667,2.0,0.0,3.333333,NaN,NaN,NaN,NaN,NaN
1,1.0,2.0,URBAN,ABIA,South East,YES,2.0,2.0,0.0,0.0,1.0,Female,45.0,Senior secondary,RENT,Fourth,Second,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,6,0,2.0,30.500000,2.0,1.0,5.500000,NaN,NaN,NaN,NaN,NaN
2,1.0,3.0,URBAN,ABIA,South East,YES,5.0,1.0,0.0,2.0,1.0,Male,41.0,Senior secondary,OWN,Fourth,Middle,NO,0.0,YES,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,5,30,1.0,34.000000,1.0,1.0,4.000000,NaN,NaN,NaN,NaN,NaN
3,1.0,4.0,URBAN,ABIA,South East,YES,4.0,1.0,1.0,0.0,0.0,Male,81.0,None,OWN,Middle,Poorest,YES,2.0,YES,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,7,144,1.0,38.000000,1.0,1.0,8.000000,1.0,33.0,1.0,5.0,0.0
4,1.0,5.0,URBAN,ABIA,South East,YES,5.0,1.0,0.0,0.0,1.0,Male,63.0,Primary,RENT,Fourth,Middle,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.61955,1.0,1.0,MICS,7,0,1.0,23.000000,1.0,1.0,4.000000,NaN,NaN,NaN,NaN,NaN


In [98]:
# Sanity check
ch_subset[(ch_subset["hh1"] == 1.0)& (ch_subset["hh2"] == 3.0 ) ]

,hh1,hh2,ln,c_mother_ln,c_mother_education,consent,survey,c_sex,c_age_yr,c_age_month,c_age_month_cat6,c_age_month_cat11,c_breastfed,c_breastfeeding,c_infant_formula,c_animal_milk,c_yogurt,c_fortified_food,c_grain,c_vitA_vegs,c_roots,c_green_leafy_vegs,c_vitA_fruits,c_other_fruits_veg,c_organ_meat,c_meat,c_eggs,c_fish,c_beans_pulse,c_cheese_f_milk,zone,area,c_mother_education,stratum,chweightmics
0,1.0,3.0,4.0,2.0,Higher/tertiary,YES,MICS,MALE,3.0,39.0,36-47,36-47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South East,URBAN,Higher/tertiary,1.0,0.658014
1,1.0,3.0,5.0,2.0,Higher/tertiary,YES,MICS,MALE,0.0,4.0,0-5,0-11,YES,YES,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,South East,URBAN,Higher/tertiary,1.0,0.658014


In [102]:
# Final variables of interest for household level analysis
final_cols = ["hh1","hh2", 
              "area", "zone", 
              "hh_members_num", 
              "num_of_women_15_49", "num_of_men_15_49",
              "num_under_5_child", "num_5_17_child",
              "hhsex", 
              "hhage", 
              "helevel",
              "hh_own_dwelling",
              
              # ====HH Agricultural variables
              "hh_agricultural_land",
              "hh_ag_land_size",
              "hh_own_animal", # own livestock
              
               # ===========Men in the household
              # Number of men who attended school
              "hh_mn_attended_sch_num",
              # average age of men
              "hh_mn_mean_age_yrs",
              # Number of men who attended at least secodn
              "hh_mn_attended_atleast_sec_sch",
              # Average life satisfaction of men in hh
              "hh_mn_mean_life_satisfaction",
              
              #========Women in household =======#
              # Number of women who attended school
              "hh_wm_attended_sch_num",
              # Average age of women
              "hh_wm_mean_age_yrs",
              # Number of women that attended at least secondary education
              "hh_wm_attended_atleast_sec_sch",
              # Average life satisfaction of women in hh
              "hh_wm_mean_life_satisfaction",

              
               # ====HH water
              "water_source",
              "other_water_source",
              "water_source_location",
              
              # ====survey weights
              "wi_quintile_mics",
              "urban_wi_quintile_mics",
              
              # === calculated livestock num and FIES
              "hh_ls_num",
              "fies_score",
            
              # === survey design parameters
              "hhweightmics",
              "psu",
              "stratum",
             ]

len(final_cols)

34

In [103]:
df = final_merged_df[final_cols]

# Save to csv
df.to_csv("../processed_data/final_df_household_fies.csv")

In [105]:
df.shape

(33631, 34)

### Life Satisfaction and Overall Happiness

* Happiness and life satisfaction at household level
* Happiness and life satisfaction at women level
* Happiness and life satisfaction at men level

In [109]:
# Men Happiness and life satisfaction
mn_happ_cols = ["hh1", "hh2","hh6", "hh7","zone",
           "ln",
           "mwb4", "mwb5",
           "mmstatus","mma1",
           "mwage",
           "mls1", "mls2", "mls3", "mls4",
           "mwelevel","windex5",
           "survey", "mwm9",
          "mnweight"]

mn_happ_dict = {
    "mwm9": "consent",
    "mls1": "mn_overall_happiness",
    "mls2": "mn_life_satisfaction",
    "mls3": "mn_life_satisfaction_last_year",
    "mls4": "mn_life_satisfaction_expectation",
    "mwb5": "mn_attended_sch",
    "mwelevel": "mn_education",
    "mmstatus": "mn_mstatus",
    "mwb4": "mn_age_yrs",
    
}


df_mn_happiness = mn[mn_happ_cols]

# Rename the columns of wm_subset using the key-value pairs in wm_dicts
df_mn_happiness = df_mn_happiness.rename(columns=mn_happ_dict)
# Save dataframe women level
df_mn_happiness.to_csv("../processed_data/men_level_happiness.csv", index=False)

In [110]:
# Women happiness and life satisfaction
wm_happ_cols = ["hh1", "hh2","hh6", "hh7", "zone", "ln",
           "wb4", "wm9",
           "wage", "wb5", "wb6a","welevel", 
           "ls1","ls2","ls3","ls4",
           "survey", "mstatus", 
           "windex5u",
           "wmweight", "stratum"]

wm_happ_dicts  = {
    "wb4": "wm_age_yrs",
    "wm9": "consent",
    "ls1": "wm_overall_happiness",
    "ls2": "wm_life_satisfaction",
    "ls3": "wm_life_satisfaction_last_year",
    "ls4": "wm_life_satisfaction_expectation",
    "wage": "wm_age_cat",
    "wb5": "wm_attended_sch",
    "wb6a": "wm_level_education",
    "mstatus": "wm_mstatus",
     # Wealth index
    "windex5": "wi_quintile_mics",

}


df_wm_happiness = wm.loc[:, wm_cols]

# Rename the columns of wm_subset using the key-value pairs in wm_dicts
df_wm_happiness = df_wm_happiness.rename(columns=wm_happ_dicts)

# Save dataframe men level
df_wm_happiness.to_csv("../processed_data/women_level_happiness.csv", index=False)

In [111]:
# Women happiness and life satisfaction
wm_happ_cols = ["hh1", "hh2","hh6", "hh7", "zone", "ln",
           "wb4", "wm9",
           "wage", "wb5", "wb6a","welevel", 
           "ls1","ls2","ls3","ls4",
           "survey", "mstatus", 
           "windex5u",
           "wmweight", "stratum"]

wm_happ_dicts  = {
    "wb4": "wm_age_yrs",
    "wm9": "consent",
    "ls1": "wm_overall_happiness",
    "ls2": "wm_life_satisfaction",
    "ls3": "wm_life_satisfaction_last_year",
    "ls4": "wm_life_satisfaction_expectation",
    "wage": "wm_age_cat",
    "wb5": "wm_attended_sch",
    "wb6a": "wm_level_education",
    "mstatus": "wm_mstatus",
     # Wealth index
    "windex5": "wi_quintile_mics",

}


df_wm_happiness = wm.loc[:, wm_cols]

# Rename the columns of wm_subset using the key-value pairs in wm_dicts
df_wm_happiness = df_wm_happiness.rename(columns=wm_happ_dicts)

# Save dataframe men level
df_wm_happiness.to_csv("../processed_data/women_level_happiness.csv", index=False)

In [112]:
# Household level happiness and life expectancy

happiness_cols = ["hh1","hh2", 
                  "area", "zone", "state",
                  "hh_members_num",
                  "hhsex", 
                  "hhage", 
                  "helevel",
                  
                 "wi_quintile_mics",
                  "urban_wi_quintile_mics",
                  "fies_score",
                  "hh_mn_mean_age_yrs",
                   "hh_wm_mean_age_yrs",
                  "hh_mn_mean_life_satisfaction",
                  "hh_mn_overall_happiness",
                  "hh_wm_mean_life_satisfaction",
                  "hh_wm_overall_happiness"
                 ]

df_hh_happiness  = final_merged_df[happiness_cols]

# Save dataframe hh level
df_hh_happiness.to_csv("../processed_data/hh_level_happiness.csv", index=False)



In [127]:
df.shape

(33631, 34)

## Summary
* The original household dateset(`hh`) comprises of 41532 dataset.
* Of these households, `33631` household didn't provide content.
* The dataset contains data from two seperate surveys (MICS and NCIS). We filter by **MICS** survey only.
* Women, men, and children data merged with the household dataset.
* After initial data preprocessing, we reduced the variables from over 300 variables to 34.


In [128]:
hh.shape

(41532, 354)